In [1]:
from datasets import Dataset
from ragas.metrics import answer_relevancy, faithfulness, context_recall, context_precision
from ragas import evaluate
import os

# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


def calculate_RAGAS_answer(generated_answer, gold_answer, question, context):
  data = {'question': [question],
          'contexts': [context],
          'answer': [generated_answer],
          'ground_truth': [gold_answer]}
  dataset = Dataset.from_dict(data)
  score = evaluate(dataset, metrics = [faithfulness, answer_relevancy, context_precision, context_recall]).to_pandas()
  faithfulness_score = score['faithfulness'].iloc[0]
  answer_relevancy_score = score['answer_relevancy'].iloc[0]
  context_precision_score = score['context_precision'].iloc[0]
  context_recall_score = score['context_recall'].iloc[0]
  return faithfulness_score, answer_relevancy_score, context_precision_score, context_recall_score

generated_answer = "The first superbowl was held on Jan 15, 1967"
gold_answer = "The first superbowl was held on Jan 15, 1967"
question = "When was the first super bowl?"
context = ["The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."]

x = calculate_RAGAS_answer(generated_answer, gold_answer, question, context)

print(x)

c:\Users\krisr\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\krisr\AppData\Local\Programs\Python\Python312\Lib\site-packages\ragas\prompt\base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.llms.prompt import PromptValue
Evaluating:  25%|██▌       | 1/4 [00

(nan, nan, nan, nan)


In [3]:
%pip install -q scikit-learn seaborn

Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn.metrics import accuracy_score, average_precision_score, classification_report, confusion_matrix, precision_score, precision_recall_fscore_support, recall_score
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import requests

In [6]:
def evaluate_multiclass_classification(y_true, y_pred, class_labels):
    """
    Evaluates a multiclass classification model.
    
    y_true: Ground truth labels
    y_pred: Predicted labels from the classifier
    class_labels: List of class names
    
    Returns a dictionary of accuracy, precision, recall, F1, and confusion matrix.
    """
    
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')  # macro-averaging
    
    # Classification report (optional detailed breakdown for each class)
    class_report = classification_report(y_true, y_pred, target_names=class_labels)
    
    # Confusion Matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # Print evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro Avg): {precision:.4f}")
    print(f"Recall (Macro Avg): {recall:.4f}")
    print(f"F1 Score (Macro Avg): {f1:.4f}")
    print("\nClassification Report:\n", class_report)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Return metrics for further analysis if needed
    return {
        "accuracy": accuracy,
        "precision_macro": precision,
        "recall_macro": recall,
        "f1_macro": f1,
        "confusion_matrix": conf_matrix
    }


In [9]:
%pip install fastapi uvicorn langchain_groq sentence-transformers

  Using cached setuptools-75.2.0-py3-none-any.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/203.1 MB ? eta -:--:--
   - -------------------------------------- 5.2/203.1 MB 26.5 MB/s eta 0:00:08
   - -------------------------------------- 7.6/203.1 MB 23.5 MB/s eta 0:00:09
   - -------------------------------------- 7.6/203.1 MB 23.5 MB/s eta 0:00:09
   - -------------------------------------- 8.4/203.1 MB 10.6 MB/s eta 0:00:19
   - -------------------------------------- 9.2/203.1 MB 9.2 MB/s eta 0:00:22
   - -------------------------------------- 10.0/203.1 MB 8.2 MB/s eta 0:00:24
   -- ------------------------------------- 10.7/203.1 MB 7.5 MB/s eta 0:00:26
   -- ------------------------------------- 11.5/203.1 MB 6.9 MB/s eta 0:00:28
   -- ------------------------------------- 12.3/203.1 MB 6.5 MB/s eta 0:00:30
   -- ------------------------------------- 13.1/203.1 MB 6.2 MB/s eta 0:00:31
   -- ------------------------------------- 13.6/203.1 MB 6.0 MB/s eta 0

In [21]:
# 1. Library imports
import pandas as pd
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import uvicorn
from IPython.display import Markdown, display
from extraction import llmAgent
from ppi_deprescribe import merge_results, ppi_deprescribe
import os 
from pathlib import Path 

app = FastAPI()
path = Path('../../data/')


@app.get('/')
def index():
    #add link to documentation
    link = 'http://127.0.0.1:8000/docs'
    content = f'''<h1>Rx Deprescribe API</h1>
            <h2>Click <a href={link}>here</a> to view the API documentation</h2>
                ''' 
    return HTMLResponse(content=content)   

@app.get('/Deprescribe')
def deprescribe(key):
    # extract information
    llm_agent = llmAgent(groq_key=groq, data_path=path)

    results_dict = {
        "diagnosis_dict": llm_agent.extract_diagnosis(encounter_key=key),
        "encounter_dict": llm_agent.extract_encounter_info(encounter_key=key),
        # Is the reasoning in the json or sepearte?
        # Should the reasoning be included in any of them or just the diangosis with the reasoning seperate?
        "notes_dict": llm_agent.extract_notes(encounter_key=key),
    }
    print(results_dict['notes_dict'])
    # # #   master formatter step   # # #
    # merge the diagnosis booleans (just use OR logic for now)
    # make a final "reasoning" behind the recommendation
    final_dict = merge_results(results_dict=results_dict)

    # feed the three reasonings to LLM to get a single summary
    final_reasoning = llm_agent.summarize_reasonings(results_dict=results_dict)

    # # #   get recommendation from PPI algorithm   # # #
    recommendation_str = ppi_deprescribe(patient_diagnosis=final_dict)
    return recommendation_str, final_reasoning
    #print("Recommendation: ")
    #print(recommendation_str)
    #print("\nReasoning: ")
    #print(final_reasoning)


In [25]:
import requests
import asyncio
import nest_asyncio

nest_asyncio.apply()

def call_deprescribe_api(input_data):
    url = 'http://localhost:8000/Deprescribe'
    response = requests.get(url, params={'key': input_data})
    return response.json()

# data = pd.read_csv('../../data/')

input_data = 'D6253A5CE371EA'

async def main():
    print(1)
    await uvicorn.run(app, host='127.0.0.1', port=8000)
# Start the FastAPI server
await main()

# Call the API
result = call_deprescribe_api(input_data)
print(result)

# what data do I pass?
# do I need to reformat it?
# do I need to change the path in the api call?



1


INFO:     Started server process [25660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25660]


TypeError: object NoneType can't be used in 'await' expression

In [14]:
# load data
ground_truth = pd.read_csv('../../data/LabeledResponses.csv')
ground_truth

# y_true = ground_truth['recommendation']
# y_pred = 
# class_labels = ['continue', 'deprescribe', 'stop']

# # Evaluate the model
# metrics = evaluate_multiclass_classification(y_true, y_pred, class_labels)

,key,reason,recommendation,GS_response,Note
0,D6253A5CE371EA,example with gastrointestinal bleed should con...,continue,This patient has a history of a gastrointestin...,NaN
1,DFAFED1811B871,example with a documented history of a GI blee...,continue,This patient has a history of a gastrointestin...,NaN
2,D4865B8BBB294E,example with GI bleeding that occurred during ...,continue,GI bleeding developed during hospital stay and...,NaN
3,D00F3A8D5F43B2,example with symptoms of heartburn (upper GI s...,deprescribe,09/20/2023 inpatient nursing note mentions use...,NaN
4,D8017C77BA15FA,14d h pylori treatment if pt treated for 2wks ...,stop,Stop pantoprazole 40mg when H. Pylori treatmen...,NaN
5,DA15CCF42ACF8B,"Barrett's esophagus, continue PPI",continue,Patient should continue lansoprazole 30mg ever...,NaN
6,D45728A2EFD315,"example with esophagitis, continue PPI",continue,Patient should continue pantoprazole based on ...,NaN
7,DFB07E6B8F0957,example with chronic NSAID use and pharmacist ...,stop,Patient should stop lansoprazole 30mg twice da...,NaN
8,D1BD3665C06499,example where pt should be on due to history o...,continue,Patinet should continue omeprazole 40mg twice ...,NaN
9,D068E26FFF9F43,example where pt came in on and no clear reaso...,stop,Patient came in on lansoprazole 30mg. No clea...,NaN
